In [1]:
import os
import pandas as pd
import numpy as np

## **Extreme Temperature:**
## **Exposure and Vulnerability**

In [2]:
# Define the airports
airports = ['Milano Malpensa', 'Milano Linate', 'Bergamo Orio al Serio', 'Roma Fiumicino', 'Roma Ciampino', 
            'Napoli Capodichino', 'Palermo Punta Raisi', 'Catania Fontanarossa', 'Cagliari Elmas']

## **Folder to save the csv file where average exposure and vulnerability are going to be stored**

In [3]:
csv_folder_path = '/work/cmcc/dg07124/climax/indicators/cordex2/vulnerability_exposure'

if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)

### **Air Land -Side Components**

In [4]:
# Define the values for each indicator and airport
air_land_side_components = {
    'Runways': [470400, 159742, 145530, 396000, 9900, 118260, 292650, 109575, 126000],
    'Taxiways': [250.5, 156, 183, 300, 301, 99, 368, 159, 604],
    'Tower': [80, 47, 30, 56, 60, 40, 25, 12, 28],
    'Apron': [1319000, 387000, 190000, 797250, None, 200000, 148000, 180000, 156000],
    'Terminals': [315000, 85050, 53025, 354300, 20950, 30700, 35400, 43310, 41290],
    'Offices_and_other_buildings': [26715, 13527, 7075, 29520, 5950, 1915, 4180, 6920, 5030],
    'Airport_accesses_systems': [31660, None, None, None, None, None, None, None, None],
    'Carparks': [15000, 3000, 8000, 20100, 1220, 1500, 1364, 1800, 2133]
}

In [5]:
# Function to normalize data while handling NaN values and ZeroDivisionError
def normalize_data(indicator_values, airports):
    normalized_data = {}

    for indicator, values in indicator_values.items():
        # Filter out None (NaN) values for normalization
        valid_values = [value for value in values if value is not None]

        # Check if we have more than one unique value for normalization
        if len(valid_values) > 1:
            # Get the min and max values for the valid values
            min_val = min(valid_values)
            max_val = max(valid_values)

            # Check for zero division (when min == max)
            if min_val == max_val:
                normalized_values = [None if value is not None else None for value in values]
            else:
                # Normalize the values
                normalized_values = [
                    (value - min_val) / (max_val - min_val) if value is not None else None
                    for value in values
                ]
        else:
            # If all values are the same or all values are NaN, set all normalized values to NaN
            normalized_values = [None if value is not None else None for value in values]

        normalized_data[indicator] = normalized_values

    return normalized_data

In [6]:
# Dictionary to store the dynamic DataFrames
dynamic_dataframes = {}

# Function to display data in a DataFrame format and store the avg_exposure with a unique name
def display_data(indicator_values, airports, df_name):
    # Create a DataFrame from the original data
    df = pd.DataFrame(indicator_values, index=airports)
    print("Original Data:")
    display(df)

    # Normalize the data
    normalized_data = normalize_data(indicator_values, airports)

    # Convert the normalized data to a DataFrame
    normalized_df = pd.DataFrame(normalized_data, index=airports)

    # Calculate the average normalized value for each airport
    normalized_df['average'] = normalized_df.mean(axis=1)
    print("Normalized Data:")
    display(normalized_df)

    # Separate the "Average" column into another DataFrame
    avg_df = normalized_df[['average']]
    print("Avg components:")
    
    # Store the avg_exposure_df_temp in the dynamic_dataframes dictionary with the provided name
    dynamic_dataframes[df_name] = avg_df
    print(f"Stored DataFrame with name: {df_name}")

In [7]:
# Display the air land side components
display_data(air_land_side_components, airports, 'avg_df_exposure')

Original Data:


,Runways,Taxiways,Tower,Apron,Terminals,Offices_and_other_buildings,Airport_accesses_systems,Carparks
Milano Malpensa,470400,250.5,80,1319000.0,315000,26715,31660.0,15000
Milano Linate,159742,156.0,47,387000.0,85050,13527,NaN,3000
Bergamo Orio al Serio,145530,183.0,30,190000.0,53025,7075,NaN,8000
Roma Fiumicino,396000,300.0,56,797250.0,354300,29520,NaN,20100
Roma Ciampino,9900,301.0,60,NaN,20950,5950,NaN,1220
Napoli Capodichino,118260,99.0,40,200000.0,30700,1915,NaN,1500
Palermo Punta Raisi,292650,368.0,25,148000.0,35400,4180,NaN,1364
Catania Fontanarossa,109575,159.0,12,180000.0,43310,6920,NaN,1800
Cagliari Elmas,126000,604.0,28,156000.0,41290,5030,NaN,2133


Normalized Data:


,Runways,Taxiways,Tower,Apron,Terminals,Offices_and_other_buildings,Airport_accesses_systems,Carparks,average
Milano Malpensa,1.000000,0.300000,1.000000,1.000000,0.882106,0.898388,None,0.729873,0.830052
Milano Linate,0.325390,0.112871,0.514706,0.204099,0.192290,0.420648,None,0.094280,0.266326
Bergamo Orio al Serio,0.294528,0.166337,0.264706,0.035867,0.096220,0.186923,None,0.359110,0.200527
Roma Fiumicino,0.838436,0.398020,0.647059,0.554441,1.000000,1.000000,None,1.000000,0.776851
Roma Ciampino,0.000000,0.400000,0.705882,NaN,0.000000,0.146169,None,0.000000,0.208675
Napoli Capodichino,0.235309,0.000000,0.411765,0.044406,0.029249,0.000000,None,0.014831,0.10508
Palermo Punta Raisi,0.614007,0.532673,0.191176,0.000000,0.043348,0.082050,None,0.007627,0.210126
Catania Fontanarossa,0.216450,0.118812,0.000000,0.027327,0.067077,0.181308,None,0.030720,0.09167
Cagliari Elmas,0.252117,1.000000,0.235294,0.006832,0.061017,0.112842,None,0.048358,0.245209


Avg components:
Stored DataFrame with name: avg_df_exposure


In [8]:
# Accessing the avg_exposure_temp DataFrames
avg_df_exposure = dynamic_dataframes['avg_df_exposure']
normalized_avg_path = os.path.join(csv_folder_path, f'avg_df_exposure.csv')
avg_df_exposure.to_csv(normalized_avg_path)
display(avg_df_exposure)

,average
Milano Malpensa,0.830052
Milano Linate,0.266326
Bergamo Orio al Serio,0.200527
Roma Fiumicino,0.776851
Roma Ciampino,0.208675
Napoli Capodichino,0.10508
Palermo Punta Raisi,0.210126
Catania Fontanarossa,0.09167
Cagliari Elmas,0.245209


### **Sensitivity**

## Calculation of Vulnerability

In [9]:
# Define the sensitivity indicators and their values
sensitivity_indicators = {
    'Soil_sealing': [1235, 300, 300, 1590, 133, 217, 391, 225, 246],
    'Passengers': [27000000, 7000000, 13857257, 43532573, 5879496, 10860068, 7018087, 10223113, 4747806],
    'Air_traffic': [201050, 85730, 95377, 309783, 52253, 82577, 54243, 73494, 39691],
    'Parking_accesses': [None, None, None, None, None, None, None, None, None],
    'Age_buildings': [73, 82, 83, 63, 105, 111, 61, 97, 84],
    'Buildings_bad_conditions_or_maintenance': [None, None, None, None, None, None, None, None, None],
    'Underground_infrastructures': [30300, 14700, 5700, 26250, 2250, 0, 0, 8650, 0],
    'Flooded_areas': [None, None, None, None, None, None, None, None, None]
}

In [10]:
# Execute the function to display the data for Sensitivity Indicators
display_data(sensitivity_indicators, airports, 'avg_df_sensitivity')

Original Data:


,Soil_sealing,Passengers,Air_traffic,Parking_accesses,Age_buildings,Buildings_bad_conditions_or_maintenance,Underground_infrastructures,Flooded_areas
Milano Malpensa,1235,27000000,201050,None,73,None,30300,None
Milano Linate,300,7000000,85730,None,82,None,14700,None
Bergamo Orio al Serio,300,13857257,95377,None,83,None,5700,None
Roma Fiumicino,1590,43532573,309783,None,63,None,26250,None
Roma Ciampino,133,5879496,52253,None,105,None,2250,None
Napoli Capodichino,217,10860068,82577,None,111,None,0,None
Palermo Punta Raisi,391,7018087,54243,None,61,None,0,None
Catania Fontanarossa,225,10223113,73494,None,97,None,8650,None
Cagliari Elmas,246,4747806,39691,None,84,None,0,None


Normalized Data:


,Soil_sealing,Passengers,Air_traffic,Parking_accesses,Age_buildings,Buildings_bad_conditions_or_maintenance,Underground_infrastructures,Flooded_areas,average
Milano Malpensa,0.756349,0.573735,0.597422,None,0.24,None,1.000000,None,0.633501
Milano Linate,0.114619,0.058069,0.170457,None,0.42,None,0.485149,None,0.249659
Bergamo Orio al Serio,0.114619,0.234872,0.206174,None,0.44,None,0.188119,None,0.236757
Roma Fiumicino,1.000000,1.000000,1.000000,None,0.04,None,0.866337,None,0.781267
Roma Ciampino,0.000000,0.029179,0.046510,None,0.88,None,0.074257,None,0.205989
Napoli Capodichino,0.057653,0.157594,0.158783,None,1.00,None,0.000000,None,0.274806
Palermo Punta Raisi,0.177076,0.058535,0.053878,None,0.00,None,0.000000,None,0.057898
Catania Fontanarossa,0.063143,0.141172,0.125154,None,0.72,None,0.285479,None,0.266989
Cagliari Elmas,0.077557,0.000000,0.000000,None,0.46,None,0.000000,None,0.107511


Avg components:
Stored DataFrame with name: avg_df_sensitivity


In [11]:
# Accessing the avg_exposure_temp DataFrames
avg_df_sensitivity = dynamic_dataframes['avg_df_sensitivity']
display(avg_df_sensitivity)

,average
Milano Malpensa,0.633501
Milano Linate,0.249659
Bergamo Orio al Serio,0.236757
Roma Fiumicino,0.781267
Roma Ciampino,0.205989
Napoli Capodichino,0.274806
Palermo Punta Raisi,0.057898
Catania Fontanarossa,0.266989
Cagliari Elmas,0.107511


## Adaptative capacity

- **Risk awareness: Initiatives for mitigation to climate change:**
 1. Adherence to "NetZero2050" Programme (neutrality3+)
 2. Certification ISO 50001 and initiatives of "energy saving"
 3. Neutrality 3+ Airport carbon accreditation
 4. Neutrality 4+ Airport carbon accreditation; EP-100 intelligent use of energy from "The Climate Group"- Certification ISO 50001
 5. Level 2 Airport carbon accreditation, environmental improvement program
 6. Commitment to obtaining certfiication

**Initiatives for adaptation to climate change:**
- **Efficient drainage system**
 1. Present

- **Insurance policy for extreme events**
 1. Absent

- **Monitoring and alarm system**
 1. Present
 2. Present (but alert wind shear)
 2. Absent
  
- **Bioinfiltration and permeable pavements**
 1. Present
 2. Absent

- **Risk awareness (and initiatives of mitigation)**
 1. Risk management framework 
 2. Development of local (Urban) climate change mitigation and adaptation strategies
 3. Sustainability Report 2019
 4. gesap environmental report
 5. Absent

- **Guidelines for adaptation plan to climate change**
 1. plan (PACC) and regional strategy for  adaptation to climate change
 2. Regional plan under development
 3. Absent
 4. Sustainable Energy and Climate Action Plan (Covenant of Mayors)
 5. Sustainable Energy and Climate Action Plan (Covenant of Mayors) municipality of Catania
 6. Regional climate change adaptation strategy

| Class No. | Description      | Value Rane | Indicator Range (0-1) |
|-----------|------------------|-------------|-----------------------|
| 1         | Optimal          | 0 - 0.2     | 0.1                   |
| 2         | Quite Positive   | 0.2 - 0.4   | 0.3                   |
| 3         | Neutral          | 0.4 - 0.6   | 0.5                   |
| 4         | Quite Negative   | 0.6 - 0.8   | 0.7                   |
| 5         | Critical         | 0.8 - 1     | 0.9                   |


In [12]:
# Define the sensitivity indicators and their values
adaptative_indicators = {
    'Vegetation_zone': [None, None, None, None, None, None, None, None, None],
    'Initiatives_mitigation_climate_change_risk_awareness': [0.3, 0.3, 0.3, 0.1, 0.1, 0.3, 0.5, 0.7, 0.7],
    'Efficient_drainage_system': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    'Insurance_policy_for_extreme_events': [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Monitoring_and_alarm_system': [0.1, 0.1, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    'Bioinfiltration_and_permeable_pavements': [0.9, 0.9, 0.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Guidelines_for_adaptation_climate_change': [0.3, 0.3, 0.3, 0.5, 0.5, 0.9, 0.3, 0.3, 0.3]
}

In [13]:
# Create the DataFrame for the adaptive indicators
adaptative_df= pd.DataFrame(adaptative_indicators, index=airports)

# Add a column with the average of all indicators for each airport
adaptative_df['average'] = adaptative_df.mean(axis=1)

display(adaptative_df)

# Extract a column the average column
avg_df_adaptative = adaptative_df[['average']]

display(avg_df_adaptative)

,Vegetation_zone,Initiatives_mitigation_climate_change_risk_awareness,Efficient_drainage_system,Insurance_policy_for_extreme_events,Monitoring_and_alarm_system,Bioinfiltration_and_permeable_pavements,Guidelines_for_adaptation_climate_change,average
Milano Malpensa,None,0.3,0.1,0.9,0.1,0.9,0.3,0.433333
Milano Linate,None,0.3,0.1,0.9,0.1,0.9,0.3,0.433333
Bergamo Orio al Serio,None,0.3,0.1,0.9,0.3,0.1,0.3,0.333333
Roma Fiumicino,None,0.1,0.1,0.9,0.3,0.9,0.5,0.466667
Roma Ciampino,None,0.1,0.1,0.9,0.3,0.9,0.5,0.466667
Napoli Capodichino,None,0.3,0.1,0.9,0.3,0.9,0.9,0.566667
Palermo Punta Raisi,None,0.5,0.1,0.9,0.3,0.9,0.3,0.5
Catania Fontanarossa,None,0.7,0.1,0.9,0.3,0.9,0.3,0.533333
Cagliari Elmas,None,0.7,0.1,0.9,0.3,0.9,0.3,0.533333


,average
Milano Malpensa,0.433333
Milano Linate,0.433333
Bergamo Orio al Serio,0.333333
Roma Fiumicino,0.466667
Roma Ciampino,0.466667
Napoli Capodichino,0.566667
Palermo Punta Raisi,0.5
Catania Fontanarossa,0.533333
Cagliari Elmas,0.533333


## **Vulnerability calculation**
Average between sensitivity and adaptitive capcity

In [14]:
# Create the vulnerability DataFrame by calculating the average of both 'Average' columns
vulnerability_df = pd.DataFrame()

# Combine the 'Average' columns from both DataFrames and calculate the new average
vulnerability_df['average'] = (avg_df_sensitivity['average'] + avg_df_adaptative['average']) / 2

normalized_avg_path = os.path.join(csv_folder_path, f'avg_df_vulnerability.csv')
vulnerability_df.to_csv(normalized_avg_path)

# Display the resulting vulnerability DataFrame
vulnerability_df

,average
Milano Malpensa,0.533417
Milano Linate,0.341496
Bergamo Orio al Serio,0.285045
Roma Fiumicino,0.623967
Roma Ciampino,0.336328
Napoli Capodichino,0.420736
Palermo Punta Raisi,0.278949
Catania Fontanarossa,0.400161
Cagliari Elmas,0.320422


### Risk Calculation for Extreme Temperature

### UERRA DATASETS:
- **Extreme Temperature**

In [24]:
hazard_uerra_temp_path = '/work/cmcc/dg07124/climax/indicators/hazards_csv/hazard_uerra_temp.csv'
hazard_uerra_temp = pd.read_csv(hazard_uerra_temp_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_uerra_temp.index = hazard_uerra_temp.index.str.title()
hazard_uerra_temp['average'] = pd.to_numeric(hazard_uerra_temp['average'], errors='coerce')
hazard_uerra_temp = hazard_uerra_temp.sort_index()
hazard_uerra_temp

,average
Airports,
Bergamo Orio Al Serio,13.090606
Cagliari Elmas,15.807716
Catania Fontanarossa,17.416528
Milano Linate,13.321168
Milano Malpensa,12.487971
Napoli Capodichino,14.670617
Palermo Punta Raisi,13.622919
Roma Ciampino,14.587898
Roma Fiumicino,12.443477


In [25]:
vulnerability_df.index = vulnerability_df.index.str.title()
vulnerability_df['average'] = pd.to_numeric(vulnerability_df['average'], errors='coerce')
vulnerability_df = vulnerability_df.sort_index()
vulnerability_df

,average
Bergamo Orio Al Serio,0.285045
Cagliari Elmas,0.320422
Catania Fontanarossa,0.400161
Milano Linate,0.341496
Milano Malpensa,0.533417
Napoli Capodichino,0.420736
Palermo Punta Raisi,0.278949
Roma Ciampino,0.336328
Roma Fiumicino,0.623967


In [26]:
avg_df_exposure.index = vulnerability_df.index.str.title()
avg_df_exposure['average'] = pd.to_numeric(avg_df_exposure['average'], errors='coerce')
avg_df_exposure = avg_df_exposure.sort_index()
avg_df_exposure

,average
Bergamo Orio Al Serio,0.830052
Cagliari Elmas,0.266326
Catania Fontanarossa,0.200527
Milano Linate,0.776851
Milano Malpensa,0.208675
Napoli Capodichino,0.105080
Palermo Punta Raisi,0.210126
Roma Ciampino,0.091670
Roma Fiumicino,0.245209


In [28]:
# Now multiply the 'average' column of each DataFrame
uerra_risk_temp = avg_df_exposure['average'] * hazard_uerra_temp['average'] * vulnerability_df['average']
uerra_risk_temp

Bergamo Orio Al Serio    3.097268
Cagliari Elmas           1.348982
Catania Fontanarossa     1.397558
Milano Linate            3.533992
Milano Malpensa          1.390048
Napoli Capodichino       0.648602
Palermo Punta Raisi      0.798499
Roma Ciampino            0.449764
Roma Fiumicino           1.903878
Name: average, dtype: float64

In [36]:
# Convert the Series to a DataFrame
uerra_risk_temp_df = pd.DataFrame(uerra_risk_temp)

# Rename the column
uerra_risk_temp_df.columns = ['Uerra Risk Temperature']
uerra_risk_temp_df

,Uerra Risk Temperature
Bergamo Orio Al Serio,3.097268
Cagliari Elmas,1.348982
Catania Fontanarossa,1.397558
Milano Linate,3.533992
Milano Malpensa,1.390048
Napoli Capodichino,0.648602
Palermo Punta Raisi,0.798499
Roma Ciampino,0.449764
Roma Fiumicino,1.903878
